# Web Scraper LinkedIn

# Library Import

In [ ]:
#We import all the relevant libraries for the project
from selenium import webdriver #allow us to initialize and run our selenium driver
from selenium.webdriver.common.by import By #gives us functions to find within the scraped elements
from bs4 import BeautifulSoup #helps us create a searchable element from the HTML
import time
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import math

# Web Scraper

In [ ]:
#We give selenium our user data and make it access LinkedIn
driver = webdriver.Chrome()
time.sleep(2.5)

driver.get("https://linkedin.com/uas/login")

username = driver.find_element(By.ID,"username")

username.send_keys("") 

password = driver.find_element(By.ID,'password')

password.send_keys('')

driver.find_element(By.XPATH,"//button[@type='submit']").click()

driver.implicitly_wait(10)

#we give the basic link to start the iteration through pages
base = 'https://www.linkedin.com/jobs/search/?currentJobId=3361173187&geoId=107025191&keywords=data%20scientist%2C%20barcelona&location=Barcelona%2C%20Catalonia%2C%20Spain&start='

action = webdriver.ActionChains(driver)

#initialize vectors to append values from the code
all_promoted = []
easy_applies = []
job_titles = []
company_titles = []
urls = []
locations = []
states = []
posting_dates = []
appl_counts = []
work_type  = []
seniorities  = []
emp_type = []
industries = []
num_employees = []
all_followers = []
python_required = []
i=0 #start at job 0

#We create our main loop to iterate through pages
while i <= 1000:
    
#we go to the first page of the job search
    driver.get(base+str(i))
    time.sleep(2)

#we scroll through the job list element
    jobs_div = driver.find_element(By.XPATH,'//*[@id="main"]/div/section[1]/div')
    scroll_position = 0
    height = driver.execute_script("return arguments[0].scrollHeight", jobs_div)

    while scroll_position <= height:
        scroll_position += 2
        driver.execute_script(f"arguments[0].scrollTo(0, {scroll_position});", jobs_div)
        
#we create the item through which we'll iterate for each page
    jobs_list = driver.find_elements(By.CSS_SELECTOR, '.jobs-search-results-list .job-card-container')
    
#we create the way for the code to end when we get to the last page
    if len(jobs_list) < 5:
        break

#we create a loop to go through each posting on the page and extract the data        
    for items in jobs_list:
#we click on the job
        clicker = items.find_element(By.CSS_SELECTOR, '.job-card-container__link')
        time.sleep(0.1)
        clicker.click()
#we create a soup from the small card to get some key data
        card_soup = BeautifulSoup(items.get_attribute('innerHTML'),'html.parser')

#we scroll down through each page so it loads and we get the entire needed data    
        job_div = driver.find_element(By.XPATH,'//*[@id="main"]/div/section[2]/div')
        scroll_position = 0
        height = driver.execute_script("return arguments[0].scrollHeight", job_div)

        while scroll_position <= height:
            scroll_position += 2
            driver.execute_script(f"arguments[0].scrollTo(0, {scroll_position});", job_div)

#we create a soup element for all the info in each job            
        main_page_soup = BeautifulSoup(driver.page_source,'html.parser')

#we get the promoted field from the card soup
        try:
            if card_soup.select_one('.job-card-container__footer-item').text:
                promoted_result = card_soup.select_one('.job-card-container__footer-item').text
            else:
                promoted_result = np.NaN
        except:
            promoted_result = np.NaN

# we get the job title            
        try:
            if main_page_soup.select_one(".jobs-unified-top-card__job-title"):
                job_titles.append(main_page_soup.select_one(".jobs-unified-top-card__job-title").text)
            else:
                job_titles.append(np.NaN)
        except:
            job_titles.append(np.NaN)

#we get the url for the job
        try:
            if card_soup.select_one('.job-card-container__link')['href']:
                urls.append('linkedin.com'+card_soup.select_one('.job-card-container__link')['href'])
            else:
                urls.append(np.NaN)
        except:
            urls.append(np.NaN)
        
# we get the company name
        try:
            if main_page_soup.select_one(".ember-view.t-black.t-normal"):
                company_titles.append(main_page_soup.select_one(".ember-view.t-black.t-normal").text.strip())
            else:
                company_titles.append(np.NaN)
        except:
            company_titles.append(np.NaN)
            
#we review if python is in the requirements            
        try:
            if main_page_soup.find('div',{'class':'jobs-box--with-cta-large'}):
                if 'python' in main_page_soup.find('div',{'class':'jobs-box--with-cta-large'}).text.lower():
                    python_required.append(True)
                else: 
                    python_required.append(False)
                    
            else:
                python_required.append(False)
        except:
            python_required.append(np.NaN)
            
# we get the location
        try:
            if main_page_soup.select_one('.jobs-unified-top-card__bullet'):
                locations.append(main_page_soup.select_one('.jobs-unified-top-card__bullet').text.strip())
            else:
                locations.append(np.NaN)
        except:
            locations.append(np.NaN)

#we get the list data that contains seniority, employment type. etc            
        try:
            if main_page_soup.select('.jobs-unified-top-card__job-insight'):
                list_data = main_page_soup.select('.jobs-unified-top-card__job-insight')
            else:
                list_data = np.NaN
        except:
            list_data= np.NaN

#we get the posting data            
        try:    
            if main_page_soup.select_one('.jobs-unified-top-card__posted-date'):
                posting_dates.append(main_page_soup.select_one('.jobs-unified-top-card__posted-date').text.strip())
            else:
                posting_dates.append(np.NaN)
        except:
            posting_dates.append(np.NaN)

#we get the companies followers            
        try:
            if main_page_soup.find('div',{'class':'artdeco-entity-lockup__subtitle ember-view t-16'}):
                all_followers.append(main_page_soup.find('div',{'class':'artdeco-entity-lockup__subtitle ember-view t-16'}).text.strip().split()[0])
            else:
                all_followers.append(np.NaN)
        except :
            all_followers.append(np.NaN)
            
#we get the company industry

        try:
            if main_page_soup.find('div',{'class':'t-14 mt5'}):
                industries.append(main_page_soup.find('div',{'class':'t-14 mt5'}).text.strip().split('\n')[0])
            else:
                industries.append(np.NaN)
        except: 
            industries.append(np.NaN)

#we get the number of employees
        try:    
            if main_page_soup.find_all('span',{'class':'jobs-company__inline-information'}):
                num_employees.append(main_page_soup.find_all('span',{'class':'jobs-company__inline-information'})[1].text.strip().split(' ')[0])
            else:
                num_employees.append(np.NaN)
        except:
            num_employees.append(np.NaN)
            
# we get the workplace type
        try:
            if main_page_soup.select_one('.jobs-unified-top-card__workplace-type'):
                workplace_type = main_page_soup.select_one('.jobs-unified-top-card__workplace-type').text.strip()
            else:
                workplace_type = np.NaN
        except:
            workplace_type = np.NaN

        work_type.append(workplace_type)

#we get the applicant count
        try:
            if main_page_soup.select_one(".jobs-unified-top-card__applicant-count"):
                applicant_count = main_page_soup.select_one(".jobs-unified-top-card__applicant-count").text.strip()[0]
            else:
                applicant_count = np.NaN

        except:
            applicant_count = np.NaN

        appl_counts.append(applicant_count)

#we go through the list to get employment and seniority
        try: 
            if list_data is not np.NaN:
                aux = list_data[0].text.strip().split('·')
            else:
                aux = np.NaN
        except:
            try:
                aux = list_data[0].text.strip()
            except:
                aux = np.NaN

        if type(aux)== list:
            if aux == np.NaN:
                employment_type = np.NaN
                seniority = np.NaN
            elif len(aux) ==2:
                employment_type = aux[0]
                seniority = aux[1]
            else:
                employment_type=aux[0]
                seniority = np.NaN
        else: 
            employment_type=np.NaN
            seniority = np.NaN

        seniorities.append(seniority)
        emp_type.append(employment_type)

# we get the state
        try:
            if driver.find_elements(By.CSS_SELECTOR, '.mt5.mb2 ul li')[-1]:
                aux = driver.find_elements(By.CSS_SELECTOR, '.mt5.mb2 ul li')[-1].text
                if aux == 'Actively recruiting':
                    state = 'On-going'
                elif aux == 'Be an early applicant':
                    state = 'Early Applications'
                else:
                    state = 'Other'
            else:
                state='Other'
        except:
            state = 'Other'

        states.append(state)

# we get the promoted label for each job        
        try:
            if promoted_result == 'Promoted':
                promoted = True
            else:
                promoted = False
        except:
            promoted = np.NaN

        all_promoted.append(promoted)

#we get the easy apply status through the buttons text
        try:
            if card_soup.find('li',{'class':"job-card-container__apply-method job-card-container__footer-item inline-flex align-items-center"}):
                apply = card_soup.find('li',{'class':"job-card-container__apply-method job-card-container__footer-item inline-flex align-items-center"}).text.strip()

                if apply == 'Easy Apply':
                    easy_apply = True
                else:
                    easy_apply = False
            else:
                easy_apply = False

        except:
            easy_apply = False

        easy_applies.append(easy_apply)
        
    i+=len(jobs_list)

result = pd.DataFrame(data={'Job Title':job_titles,
                            'Company Name':company_titles,
                            'State':states,
                            'Location':locations,
                            'Workspace':work_type,
                            'Posting Date':posting_dates,
                            'Seniority Level':seniorities,
                            'Industry':industries,
                            'Applicants':appl_counts,
                            'Employment Type':emp_type,
                            'Apply Through LinkedIn':easy_applies,
                            'Offer URL': urls,
                            'Promoted':all_promoted,
                            'Python Required':python_required,
                            'Employees':num_employees,
                            'Followers':all_followers})


In [ ]:
result.to_csv('result.csv')

In [ ]:
result